In [126]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

df = pd.read_csv('../data/text_data.csv')

df

,Тематика,Текст обращения,Количество комментариев,Количество репостов
0,Здравоохранение,NaN,61,50
1,Здравоохранение,Прошу рассмотреть вопрос о ремонте крыши в мно...,92,43
2,Культура,"Транспорт ходит с большими задержками, особенн...",14,43
3,Транспорт,Погибли бездомные животные из-за отравления. К...,94,28
4,Безопасность,Где обещанная модернизация больницы? Оборудова...,77,5
...,...,...,...,...
495,Образование,"Нет воды в доме номер 12 уже вторые сутки, про...",73,44
496,NaN,Прошу рассмотреть вопрос о ремонте крыши в мно...,69,46
497,Культура,Примите меры! В школе не хватает учебников для...,25,45
498,NaN,"Глава региона, прошу рассмотреть вопрос о стро...",38,0


In [127]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Тематика                 479 non-null    object
 1   Текст обращения          460 non-null    object
 2   Количество комментариев  500 non-null    int64 
 3   Количество репостов      500 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 15.8+ KB


In [128]:
df.describe()

,Количество комментариев,Количество репостов
count,500.000000,500.000000
mean,55.186000,30.428000
std,44.825909,33.133374
min,0.000000,0.000000
25%,25.000000,11.000000
50%,50.000000,24.500000
75%,77.000000,39.000000
max,300.000000,200.000000


In [129]:
df.isnull().sum().sort_values(ascending=False)

Текст обращения            40
Тематика                   21
Количество комментариев     0
Количество репостов         0
dtype: int64

In [130]:
df.duplicated().sum()

np.int64(0)

In [131]:
df['Тематика'] = df['Тематика'].fillna('не указано')
df = df.dropna(subset=['Текст обращения']).copy()
df.reset_index(drop=True, inplace=True)

df.isnull().sum().sort_values(ascending=False)

df.to_csv('../data/text_data_Cleaned.csv', index=False)

In [132]:
def preprocess_text(text):
    if pd.isna(text) or text.strip() == "":  
        return "текст отсутствует"

    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^\w\s,.!?:;]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\s+", " ", text).strip()

    stop_words = {"и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", "его",
                  "но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", "вот", "от",
                  "меня", "еще", "нет", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже",
                  "или", "ни", "быть", "был", "него", "до", "вас", "нибудь", "опять", "уж", "вам", "ведь", "там",
                  "потом", "себя", "ничего", "ей", "может", "они", "тут", "где", "есть", "надо", "ней", "для", "мы",
                  "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", "раз", "тоже", "себе", "под",
                  "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", "совсем", "ним", "здесь",
                  "этом", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", "куда", "зачем", "всех", "никогда",
                  "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", "над", "больше", "тот", "через",
                  "эти", "нас", "про", "всего", "них", "какая", "много", "разве", "три", "эту", "моя", "впрочем",
                  "хорошо", "свою", "этой", "перед", "иногда", "лучше", "чуть", "том", "нельзя", "такой", "им", "более",
                  "всегда", "конечно", "всю", "между"}
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return " ".join(words)

df["Текст обращения"] = df["Текст обращения"].apply(preprocess_text)

df

,Тематика,Текст обращения,Количество комментариев,Количество репостов
0,Здравоохранение,прошу рассмотреть вопрос ремонте крыши многокв...,92,43
1,Культура,"транспорт ходит большими задержками, особенно ...",14,43
2,Транспорт,погибли бездомные животные изза отравления. от...,94,28
3,Безопасность,обещанная модернизация больницы? оборудование ...,77,5
4,ЖКХ,"уважаемая администрация, прошу обратить вниман...",11,3
...,...,...,...,...
455,Образование,"воды доме номер вторые сутки, прошу срочно раз...",73,44
456,не указано,прошу рассмотреть вопрос ремонте крыши многокв...,69,46
457,Культура,примите меры! школе хватает учебников учеников...,25,45
458,не указано,"глава региона, прошу рассмотреть вопрос строит...",38,0


In [133]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df["Текст обращения"])

num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df["Кластер"] = kmeans.fit_predict(X)

print(df["Кластер"].value_counts())

df.head()

Кластер
1    148
4     90
2     89
3     79
0     54
Name: count, dtype: int64


,Тематика,Текст обращения,Количество комментариев,Количество репостов,Кластер
0,Здравоохранение,прошу рассмотреть вопрос ремонте крыши многокв...,92,43,3
1,Культура,"транспорт ходит большими задержками, особенно ...",14,43,1
2,Транспорт,погибли бездомные животные изза отравления. от...,94,28,0
3,Безопасность,обещанная модернизация больницы? оборудование ...,77,5,1
4,ЖКХ,"уважаемая администрация, прошу обратить вниман...",11,3,4


In [134]:
for cluster in sorted(df["Кластер"].unique()):
    print(f"📌 Примеры обращений из кластера {cluster}:")
    print(df[df["Кластер"] == cluster]["Текст обращения"].head(5).tolist())
    print("-" * 50)

📌 Примеры обращений из кластера 0:
['погибли бездомные животные изза отравления. отвечает это?', 'погибли бездомные животные изза отравления. отвечает это?', 'погибли бездомные животные изза отравления. отвечает это?', 'погибли бездомные животные изза отравления. отвечает это?', 'погибли бездомные животные изза отравления. отвечает это?']
--------------------------------------------------
📌 Примеры обращений из кластера 1:
['транспорт ходит большими задержками, особенно вечернее время.', 'обещанная модернизация больницы? оборудование устарело, врачи справляются.', 'примите меры! школе хватает учебников учеников х классов.', 'обещанная модернизация больницы? оборудование устарело, врачи справляются.', 'обещанная модернизация больницы? оборудование устарело, врачи справляются.']
--------------------------------------------------
📌 Примеры обращений из кластера 2:
['крик души! поликлинике номер огромные очереди, невозможно попасть врачу.', 'крик души! поликлинике номер огромные очереди, н

In [135]:
cluster_names = {
    0: "Строительство",
    1: "Образование (нехватка учебников)",
    2: "Состояние дорог",
    3: "Поликлиники (большие очереди)",
    4: "Разные обращения"
}

df["Тематика"] = df["Кластер"].map(cluster_names)
df.drop(columns=["Кластер"], inplace=True)
df.head()

,Тематика,Текст обращения,Количество комментариев,Количество репостов
0,Поликлиники (большие очереди),прошу рассмотреть вопрос ремонте крыши многокв...,92,43
1,Образование (нехватка учебников),"транспорт ходит большими задержками, особенно ...",14,43
2,Строительство,погибли бездомные животные изза отравления. от...,94,28
3,Образование (нехватка учебников),обещанная модернизация больницы? оборудование ...,77,5
4,Разные обращения,"уважаемая администрация, прошу обратить вниман...",11,3


In [136]:
df.isna().sum()

Тематика                   0
Текст обращения            0
Количество комментариев    0
Количество репостов        0
dtype: int64

In [137]:
df[df["Тематика"] == ""]

,Тематика,Текст обращения,Количество комментариев,Количество репостов


In [138]:
df["Тематика"].unique()

array(['Поликлиники (большие очереди)',
       'Образование (нехватка учебников)', 'Строительство',
       'Разные обращения', 'Состояние дорог'], dtype=object)

In [139]:
df.duplicated().sum()

np.int64(1)

In [140]:
df[df.duplicated()]

,Тематика,Текст обращения,Количество комментариев,Количество репостов
219,Образование (нехватка учебников),примите меры! школе хватает учебников учеников...,41,18


In [141]:
df = df.drop_duplicates()

In [142]:
df.to_csv('../data/text_data_Cleaned.csv', index=False)